<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_3_tuning_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [6]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0   1051      0 --:--:-- --:--:-- --:--:--  1051
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [7]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-3.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-3.validation.csv",header=None)
valid.head()

,0,1
0,2,left heel pain
1,2,right heel became painful
2,8,Arthritis
3,8,arthritis
4,8,arthritis


In [13]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-3.test.csv",header=None)
test.head()

,0,1
0,2,heel pain
1,5,rapid heart beat
2,6,diastolic dysfunction
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.850651,4.221541,0.215466,03:36


In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.save_encoder('ft_fold3_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold3_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.030801,3.725246,0.375318,01:04


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.940539,3.197728,0.461832,01:21


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.545921,2.823537,0.526718,03:25


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.116319,2.494101,0.578880,03:17
1,1.916590,2.322519,0.604326,03:14


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.752103,2.122109,0.646310,03:28
1,1.538968,2.026263,0.664122,03:14


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.466237,1.916616,0.676845,03:27
1,1.244783,1.873456,0.692112,03:27


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.232921,1.782693,0.692112,03:31
1,1.097540,1.733699,0.708651,03:26


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.094466,1.700311,0.717557,03:16
1,0.977428,1.639786,0.720102,03:09


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.913087,1.664388,0.713740,03:13
1,0.933868,1.597586,0.726463,03:15
2,0.863074,1.554524,0.745547,03:26
3,0.740650,1.542800,0.741730,03:24


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.750738,1.562355,0.739186,03:26
1,0.834756,1.547257,0.748092,03:23
2,0.735838,1.486269,0.754453,03:25
3,0.651414,1.469710,0.759542,03:07


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.657567,1.536825,0.753181,03:06
1,0.696699,1.507825,0.750636,03:17
2,0.613127,1.503188,0.764631,03:12
3,0.573504,1.488379,0.765903,03:16


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.606077,1.550034,0.755725,03:30
1,0.640330,1.573626,0.758270,03:16
2,0.591227,1.537792,0.769720,03:29
3,0.552240,1.516325,0.773537,03:16


In [0]:
# save the best model

learn.save_encoder('ask-3-tuning-cadec')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [0]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 334, 6, 8, 8, 8, 12, 957, 1035, 27, 27, 291, 29, 30, 31, 34, 877, 597, 37, 37, 37, 37, 37, 37, 896, 37, 37, 49, 301, 49, 1031, 49, 82, 49, 49, 49, 49, 49, 49, 49, 53, 53, 53, 597, 61, 61, 491, 159, 337, 1031, 70, 71, 107, 8, 76, 209, 260, 289, 402, 633, 633, 633, 815, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 523, 289, 614, 614, 289, 80, 80, 80, 137, 218, 1031, 815, 453, 94, 453, 453, 864, 453, 453, 453, 453, 453, 453, 453, 453, 94, 94, 761, 761, 98, 98, 285, 100, 100, 100, 100, 861, 1031, 644, 877, 108, 289, 108, 289, 289, 289, 189, 117, 775, 119, 436, 815, 1031, 125, 125, 125, 125, 125, 125, 125, 126, 127, 82, 343, 343, 131, 139, 139, 139, 140, 226, 447, 151, 758, 218, 626, 441, 453, 453, 167, 473, 819, 604, 172, 172, 700, 172, 700, 427, 729, 33, 182, 182, 289, 289, 183, 183, 183, 758, 758, 758, 758, 758, 758, 758, 187, 187, 187, 187, 187, 187, 359, 289, 289, 190, 289, 703, 790, 105, 105, 105, 1

In [46]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

515
0.5946882217090069
